In [1]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import numpy as np
import random
import functools
import math
import sys
import pandas as pd
import json
import codecs
from pprint import pprint
import nltk


In [2]:
train = pd.read_csv('./en_train.csv', sep=',', encoding='utf8',
    dtype={'sentence_id': np.int32, 'token_id': np.int32, 'class': unicode, 'before': unicode, 'after': unicode})
train.dropna(axis=0, how='any', inplace=True)
train['before'] = train.before.astype(unicode)
train['after'] = train.after.astype(unicode)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9918390 entries, 0 to 9918440
Data columns (total 5 columns):
sentence_id    int32
token_id       int32
class          object
before         object
after          object
dtypes: int32(2), object(3)
memory usage: 378.4+ MB


In [3]:

class DataOperations:
    
    def __init__(self, char_embedding_size, char_to_id, char_embeddings, start_char_id, end_char_id,
                  word_embedding_size, word_embeddings, word_one_hots, word_to_id, id_to_word,
                  start_word_id, end_word_id):
        self._char_embedding_size = char_embedding_size
        self._char_to_id = char_to_id
        self._char_embeddings = char_embeddings
        self._start_char_id = start_char_id
        self._end_char_id = end_char_id
        self._word_embedding_size = word_embedding_size
        self._word_embeddings = word_embeddings
        self._word_one_hots = word_one_hots
        self._word_to_id = word_to_id
        self._id_to_word = id_to_word
        self._start_word_id = start_word_id
        self._end_word_id = end_word_id
 

    @classmethod
    def create_from_data(cls, char_data, word_data, char_embedding_size=64, word_embedding_size=64):
        # Initialize character data
        char_to_id = {}
        for text in char_data:
            for c in text:
                if c not in char_to_id:
                    id = len(char_to_id)
                    char_to_id[c] = id
        vocab_size = len(char_to_id) + 2
        char_embeddings = np.random.random([vocab_size, char_embedding_size])
        start_char_id = vocab_size - 2
        end_char_id = vocab_size - 1

        # Initialize word data
        word_to_id = {}
        id_to_word = {}
        for text in word_data:
            for word in nltk.word_tokenize(text):
                if word not in word_to_id:
                    id = len(word_to_id)
                    word_to_id[word] = id
                    id_to_word[id] = word
        vocab_size = len(word_to_id) + 2 # Add two extra 'pseudo words': start and end
        word_one_hots = np.eye(vocab_size)
        word_embeddings = np.random.random([vocab_size, word_embedding_size])
        start_word_id = vocab_size - 2
        end_word_id = vocab_size - 1

        return cls(char_embedding_size, char_to_id, char_embeddings, start_char_id, end_char_id, 
                   word_embedding_size, word_embeddings, word_one_hots, word_to_id, id_to_word,
                   start_word_id, end_word_id)


    @classmethod
    def load_from_file(cls, file_path):
        with codecs.open(file_path, encoding='utf-8') as f:    
            data = json.load(f)

            char_embedding_size = data['char_embedding_size']
            char_to_id = data['char_to_id']
            char_embeddings = data['char_embeddings']
            start_char_id = data['start_char_id']
            end_char_id = data['end_char_id']
            word_embedding_size = data['word_embedding_size']
            word_embeddings = data['word_embeddings']
            word_one_hots = data['word_one_hots']
            word_to_id = data['word_to_id']
            id_to_word = data['word_to_id']
            start_word_id = data['id_to_word']
            end_word_id = data['end_word_id']

        return cls(char_embedding_size, char_to_id, char_embeddings, start_char_id, end_char_id,
                   word_embedding_size, word_embeddings, word_one_hots, word_to_id, id_to_word,
                   start_word_id, end_word_id)



    def save(self, file_path):
        data = {}
        data['char_embedding_size'] = self._char_embedding_size
        data['char_to_id'] = self._char_to_id
        data['char_embeddings'] = self._char_embeddings.tolist()
        data['start_char_id'] = self._start_char_id
        data['end_char_id'] = self._end_char_id
        data['word_embedding_size'] = self._word_embedding_size
        data['word_embeddings'] = self._word_embeddings.tolist()
        data['word_one_hots'] = self._word_one_hots.tolist()
        data['word_to_id'] = self._word_to_id
        data['id_to_word'] = self._id_to_word
        data['start_word_id'] = self._start_word_id
        data['end_word_id'] = self._end_word_id

        with codecs.open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f)


In [4]:
char_data = train.loc[train['class'] == 'DATE', 'before']
word_data = train.loc[train['class'] == 'DATE', 'after']
opers = DataOperations.create_from_data(char_data, word_data)


In [32]:
train['class'].unique()


array([u'PLAIN', u'PUNCT', u'DATE', u'LETTERS', u'CARDINAL', u'VERBATIM',
       u'DECIMAL', u'MEASURE', u'MONEY', u'ORDINAL', u'TIME',
       u'ELECTRONIC', u'DIGIT', u'FRACTION', u'TELEPHONE', u'ADDRESS'], dtype=object)

In [39]:
pd.set_option('display.max_colwidth', 200)

In [40]:
train[train['class'] == 'FRACTION'][['before', 'after']]

,before,after
5439,1/2007,one two thousand seventh
13765,12639/12640,twelve thousand six hundred thirty nine twelve thousand six hundred fortieths
23890,10/59,ten fifty ninths
24795,11/12,eleven twelfths
25462,5/7,five sevenths
35594,11/2013,eleven two thousand thirteenths
35602,95/2005,ninety five two thousand fifths
42785,37 / 38,thirty seven thirty eighths
43266,451/452,four hundred fifty one four hundred fifty seconds
53051,7½,seven and a half


In [5]:
opers.save('./date_opers.json')

In [6]:
opers1 = DataOperations.load_from_file('./date_opers.json')
print opers1._char_embedding_size
pprint(opers1._char_to_id)
pprint(opers1._word_to_id)

64
{u' ': 6,
 u"'": 43,
 u',': 17,
 u'-': 39,
 u'.': 42,
 u'/': 33,
 u'0': 1,
 u'1': 12,
 u'2': 0,
 u'3': 18,
 u'4': 5,
 u'5': 28,
 u'6': 2,
 u'7': 3,
 u'8': 4,
 u'9': 23,
 u'A': 13,
 u'B': 44,
 u'C': 45,
 u'D': 32,
 u'E': 51,
 u'F': 34,
 u'G': 59,
 u'H': 56,
 u'I': 55,
 u'J': 36,
 u'L': 48,
 u'M': 7,
 u'N': 29,
 u'O': 37,
 u'P': 54,
 u'R': 52,
 u'S': 24,
 u'T': 40,
 u'U': 47,
 u'V': 58,
 u'W': 46,
 u'Y': 49,
 u'a': 8,
 u'b': 27,
 u'c': 10,
 u'd': 41,
 u'e': 25,
 u'f': 57,
 u'g': 20,
 u'h': 11,
 u'i': 15,
 u'j': 50,
 u'l': 16,
 u'm': 26,
 u'n': 38,
 u'o': 30,
 u'p': 14,
 u'r': 9,
 u's': 21,
 u't': 22,
 u'u': 19,
 u'v': 31,
 u'w': 53,
 u'y': 35}
{u'a': 91,
 u'april': 11,
 u'august': 15,
 u'b': 81,
 u'c': 82,
 u'd': 92,
 u'december': 24,
 u'e': 95,
 u'eight': 4,
 u'eighteen': 36,
 u'eighteens': 88,
 u'eighteenth': 52,
 u'eighth': 62,
 u'eighties': 73,
 u'eights': 94,
 u'eighty': 17,
 u'eleven': 59,
 u'elevens': 99,
 u'eleventh': 56,
 u'f': 106,
 u'february': 34,
 u'fifteen': 21,
 u'fifte

In [7]:

data = train[train['class'] == 'DATE']
oper = DataOperations.create_from_data(data['before'], data['after'])
oper.save('./date_oper.json')

In [76]:
tf.reset_default_graph()

class TextGenerator:

    def __init__(self, char_embedding_size, char_embeddings, char_to_id, start_char_id, end_char_id,
                  word_embedding_size, word_embeddings, word_one_hots, word_to_id, id_to_word, start_word_id,
                  end_word_id, in_seq_len=100, out_seq_len=50, state_size=64, num_layers=3):
        # General parameters
        self._state_size = state_size
        self._num_layers = num_layers
        self._learning_rate = .003
        self._max_grad_norm = .5

        # Character specific parameters
        self._in_seq_len = in_seq_len
        self._char_embedding_size = char_embedding_size
        self._char_embeddings = char_embeddings
        self._char_to_id = char_to_id
        self._start_char_id = start_char_id
        self._end_char_id = end_char_id

        # Word specific parameters
        self._out_seq_len = out_seq_len
        self._word_embedding_size = word_embedding_size
        self._word_embeddings = word_embeddings
        self._word_one_hots = word_one_hots
        self._word_to_id = word_to_id
        self._id_to_word = id_to_word
        self._start_word_id = start_word_id
        self._end_word_id = end_word_id
        self._num_words = self._word_one_hots.shape[1]

        # Placeholders
        self._char_input = tf.placeholder(tf.float32, [None, self._in_seq_len, self._char_embedding_size])
        self._word_input = tf.placeholder(tf.float32, [None, self._out_seq_len, self._word_embedding_size])
        self._target = tf.placeholder(tf.float32, [None, self._out_seq_len, self._num_words])

        # Graph definition
        self._init_in_length()
        self._init_out_length()
        self._init_prediction()
        self._init_cost()
        self._init_error()
        self._init_optimize()


    def _init_in_length(self):
        used = tf.sign(tf.reduce_max(tf.abs(self._char_input), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        self._in_length = tf.cast(length, tf.int32)


    def _init_out_length(self):
        used = tf.sign(tf.reduce_max(tf.abs(self._word_input), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        self._out_length = tf.cast(length, tf.int32)


    def _init_prediction(self):
        # Batch size
        batch_size = tf.shape(self._char_input)[0]

        # Input recurrent network, we don't use state
        with tf.variable_scope('input_rnn'):
            in_cells = []
            for _ in range(self._num_layers):
                in_cells.append(tf.contrib.rnn.GRUCell(self._state_size))
            in_cell = tf.contrib.rnn.MultiRNNCell(in_cells)

            in_output, _ = tf.nn.dynamic_rnn(in_cell, self._char_input, dtype=tf.float32,
                                             sequence_length=self._in_length)

        # State layer
        with tf.variable_scope('state'):
            # Output of each cell connects to state vector
            flat = tf.reshape(in_output, [-1, self._state_size])
            
            in_weight = tf.get_variable('in_W', [self._state_size, self._state_size])
            in_bias = tf.get_variable('in_b', [self._state_size], initializer=tf.constant_initializer(0.1))
            multi_result = tf.matmul(flat, in_weight)
            multi_result = tf.reshape(multi_result, [-1, self._in_seq_len, self._state_size])
            multi_result = tf.add_n(tf.split(multi_result, self._in_seq_len, axis=1))
            self._state = tf.nn.relu(multi_result + in_bias)
            self._state = tf.reshape(self._state, [-1, self._state_size])

        # Output recurrent network
        with tf.variable_scope('output_rnn'):
            #out_cells = []
            #initial_state = []
            #for _ in range(self._num_layers):
            #    out_cells.append(tf.contrib.rnn.GRUCell(self._state_size))
            #    initial_state.append(self._state)
            #out_cell = tf.contrib.rnn.MultiRNNCell(out_cells)
            
            out_cell = tf.contrib.rnn.GRUCell(self._state_size)
            self._initial_state = self._state

            out_output, self._final_state = tf.nn.dynamic_rnn(out_cell, self._word_input,
                                              dtype=tf.float32, sequence_length=self._out_length,
                                              initial_state=self._initial_state)
        
        # Softmax output
        with tf.variable_scope('softmax'): 
            out_weight = tf.get_variable('out_W', [self._state_size, self._num_words])
            out_bias = tf.get_variable('out_b', [self._num_words], initializer=tf.constant_initializer(0.1))
            out_output = tf.reshape(out_output, [-1, self._state_size])
            self._raw_logits = tf.matmul(out_output, out_weight) + out_bias
            self._logits = tf.nn.softmax(self._raw_logits)
            self._prediction = tf.reshape(self._logits, [-1, self._out_seq_len, self._num_words])

        
    def _init_cost(self):
        with tf.variable_scope('cost'):
            # Compute cross entropy for each frame.
            cross_entropy = self._target * tf.log(self._prediction + 1e-20)
            cross_entropy = -tf.reduce_sum(cross_entropy, reduction_indices=2)
            mask = tf.sign(tf.reduce_max(tf.abs(self._target), reduction_indices=2))
            cross_entropy *= mask

            # Average over actual sequence lengths.
            cross_entropy = tf.reduce_sum(cross_entropy, reduction_indices=1)
            cross_entropy /= (tf.cast(self._out_length, tf.float32) + 1e-20)

            self._cost = tf.reduce_mean(cross_entropy) + 1e-20
            tf.summary.scalar('cost', self._cost)


    def _init_error(self):
        with tf.variable_scope('error'):
            mistakes = tf.not_equal(tf.argmax(self._target, 2), tf.argmax(self._prediction, 2))
            mistakes = tf.cast(mistakes, tf.float32)
            mask = tf.sign(tf.reduce_max(tf.abs(self._target), reduction_indices=2))
            mistakes *= mask

            # Average over actual sequence lengths.
            mistakes = tf.reduce_sum(mistakes, reduction_indices=1)
            mistakes /= (tf.cast(self._out_length, tf.float32) + 1e-20)
            self._error = tf.reduce_mean(mistakes)
            tf.summary.scalar('error', self._error)

        
    def _init_optimize(self):
        tvars = tf.trainable_variables()
        grads = tf.gradients(self._cost, tvars)
        clip_grads, _ = tf.clip_by_global_norm(grads, self._max_grad_norm)
        optimizer = tf.train.AdamOptimizer(self._learning_rate)

        self._optimize = optimizer.apply_gradients(zip(clip_grads, tvars))


    def _get_batch(self, char_data, word_data):
        if len(char_data) != len(word_data):
            raise Exception('Character data length does not equal to word data'' one')
        char_input = []
        word_input = []
        word_output = []
        for i in range(len(char_data)):
            # Generate character input
            char_text = char_data[i]
            char_seq = [[0] * self._char_embedding_size for _ in range(self._in_seq_len)]
            index = 0
            if len(char_text) > (self._in_seq_len - 2):
                char_text = char_text[:self._in_seq_len - 2]

            char_seq[index] = self._char_embeddings[self._start_char_id]
            index += 1
            for c in char_text:
                if c not in self._char_to_id:
                    raise Exception('unknown symbol: ', c, ' in text: ', char_text)
                char_seq[index] = self._char_embeddings[self._char_to_id[c]]
                index += 1
            char_seq[index] = self._char_embeddings[self._end_char_id]
            char_input.append(np.array(char_seq))
            
            # Generate word input and output
            word_text = word_data[i]
            words = nltk.word_tokenize(word_text)
            if len(words) > (self._out_seq_len - 2):
                raise Exception('to long output text: ', word_text)
            elif len(words) == 0:
                raise Exception('zero output text: ', word_text)
            word_input_seq = [[0] * self._word_embedding_size for _ in range(self._out_seq_len)]
            word_target_seq = [[0] * self._word_one_hots.shape[1] for _ in range(self._out_seq_len)]
            input_index = 0
            target_index = 0
            word_input_seq[input_index] = self._word_embeddings[self._start_word_id]
            input_index += 1
            for word in words:
                if word not in self._word_to_id:
                    raise Exception('unknown word: ', word, ' in text: ', word_text)
                word_id = self._word_to_id[word]
                word_input_seq[input_index] = self._word_embeddings[word_id]
                word_target_seq[target_index] = self._word_one_hots[word_id]
                input_index += 1
                target_index += 1
            word_input_seq[input_index] = self._word_embeddings[self._end_word_id]
            word_target_seq[target_index] = self._word_one_hots[self._end_word_id]
            word_input.append(np.array(word_input_seq))
            word_output.append(np.array(word_target_seq))

        return (np.array(char_input), np.array(word_input), np.array(word_output))
    

    def sample(self, sess, char_input):
        state = sess.run(self._initial_state, feed_dict={self._char_input: char_input})

        word_input = []
        word_input_seq = []
        for _ in range(self._out_seq_len):
            word_input_seq.append([0] * self._word_embedding_size)
        word_input_seq[0] = self._word_embeddings[self._start_word_id]

        for _ in range(char_input.shape[0]):
            word_input.append(np.copy(word_input_seq))
        word_input = np.array(word_input)
        
        state, prediction = sess.run([self._final_state, self._prediction],
                                     feed_dict={self._word_input: word_input, self._state: state})
        
        ends = [False] * prediction.shape[0]
        texts = ['' for _ in range(prediction.shape[0])]
        for i in range(self._out_seq_len - 1):
            preds = np.argmax(prediction, 2)
            for j in range(preds.shape[0]):
                if ends[j]:
                    next

                word_id = preds[j][0]
                if word_id == self._end_word_id or word_id == self._start_word_id:
                    ends[j] = True
                    word_input[j] = np.copy(word_input_seq)
                else:
                    if len(texts[j]) > 0:
                        texts[j] += ' '
                    texts[j] += self._id_to_word[word_id]
                    word_input[j][0] = np.copy(self._word_embeddings[word_id])

            state, prediction = sess.run([self._final_state, self._prediction],
                                     feed_dict={self._word_input: word_input, self._state: state})

        return np.array(texts)
        

    def train(self, log_dir, model_dir, num_steps, train, validation):
        sess = tf.Session()
        
        gen_error = tf.Variable(1., name='gen_error')
        tf.summary.scalar('gen_error', gen_error)
        summaries = tf.summary.merge_all()
        
        sess.run(tf.global_variables_initializer())

        train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
        test_writer = tf.summary.FileWriter(log_dir + '/test', sess.graph)

        train_writer.add_graph(sess.graph)
        saver = tf.train.Saver()

        error = None
        for step in range(num_steps):
            train_sample = train.sample(n=100)
            char_data = np.array(train_sample['before'])
            word_data = np.array(train_sample['after'])
            
            char_input, word_input, target = self._get_batch(char_data, word_data)
            _, s = sess.run([self._optimize, summaries],
                    feed_dict={self._char_input: char_input, self._word_input: word_input, self._target: target})
            train_writer.add_summary(s, step)
            train_writer.flush()

            validation_sample = validation.sample(n=100)
            char_data = np.array(validation_sample['before'])
            word_data = np.array(validation_sample['after'])
            char_input, word_input, target = self._get_batch(char_data, word_data)

            texts = self.sample(sess, char_input)
            
            print '\n{0} --> {1} <==> {2}\n'.format(char_data[0], texts[0].strip(), word_data[0].strip())

            equals = 0.
            for i in range(len(texts)):
                  if texts[i].strip() == word_data[i].strip():
                    equals += 1
            ge = 1. - equals / len(texts)

            sess.run(gen_error.assign(ge))
            s, e = sess.run([summaries, self._error],
                    feed_dict={self._char_input: char_input, self._word_input: word_input, self._target: target})

            test_writer.add_summary(s, step)
            test_writer.flush()

            sys.stdout.write('\r{0} {1} {2}'.format(step, e, ge))

            if error is None or error > e:
                saver.save(sess, model_dir + '/best.chkp')
                error = e
                print 'save best model for: ', error
            

In [77]:
!rm -rf '/tmp/text_generation'

model = TextGenerator(oper._char_embedding_size, oper._char_embeddings, oper._char_to_id, oper._start_char_id,
                      oper._end_char_id, oper._word_embedding_size, oper._word_embeddings,
                      oper._word_one_hots, oper._word_to_id, oper._id_to_word, oper._start_word_id,
                      oper._end_word_id)

model.train('/tmp/text_generation', './text_generator', 10000, data[:200000], data[200000:])


February 2013 --> november november november november november thirteens thirteens thirteens thirteens thirteens thirteens thirteens thirteens thirteens thirteens thirteens six thirteens thirteens thirteens six thirteens thirteens thirteens six thirteens thirteens thirteens six thirteens thirteens thirteens six thirteens thirteens thirteens six thirteens thirteens thirteens six thirteens thirteens thirteens six thirteens thirteens thirteens six <==> february twenty thirteen

0 0.805190563202 1.0save best model for:  0.805191

10 April 987 --> july july july july july six six six six nine six six nine six nine six nine six nine six nine six nine six <==> the tenth of april nine eighty seven

1 0.66156744957 1.0save best model for:  0.661567

1999 -->  <==> nineteen ninety nine

2 0.604103147984 1.0save best model for:  0.604103

January 23, 2002 --> nineteen <==> january twenty third two thousand two

3 0.621575295925 1.0
1997 -->  <==> nineteen ninety seven

4 0.624484121799 1.0
1983 

174 0.339658737183 0.95
1985 --> nineteen eighty eight <==> nineteen eighty five

175 0.335825413465 0.95
2002 --> two thousand <==> two thousand two

176 0.303666681051 0.94
2004 --> two thousand <==> two thousand four

177 0.296678572893 0.94
2009 --> two thousand <==> two thousand nine

178 0.308503985405 0.94
2011-01-15 --> the twenty of of twenty fourteen <==> the fifteenth of january twenty eleven

179 0.286019831896 0.9
2014 --> twenty eleven <==> twenty fourteen

180 0.311357140541 0.88
November 2001 --> november twenty fourteen <==> november two thousand one

181 0.295329362154 0.94
1949 --> nineteen ninety <==> nineteen forty nine

182 0.31704762578 0.98
1974 --> nineteen sixty <==> nineteen seventy four

183 0.29638889432 0.96
1934 --> nineteen sixty <==> nineteen thirty four

184 0.320603191853 0.94
2015 --> twenty fifteen <==> twenty fifteen

185 0.296476215124 0.95
1979 --> nineteen seventy two <==> nineteen seventy nine

186 0.287829369307 0.94
24 May 2012 --> the twenty

267 0.263757944107 0.88
1973 --> nineteen seventy six <==> nineteen seventy three

268 0.261813521385 0.85
80's --> two thousand six <==> eighties

269 0.266257941723 0.86
2008 --> two thousand six <==> two thousand eight

270 0.254738092422 0.85
June 1991 --> august nineteen ninety <==> june nineteen ninety one

271 0.25258731842 0.9
1930 --> nineteen forty <==> nineteen thirty

272 0.233123034239 0.84
13 December 1907 --> the sixteenth of april twenty ten <==> the thirteenth of december nineteen o seven

273 0.245984122157 0.82
11 June 2007 --> the tenth of april two thousand nine <==> the eleventh of june two thousand seven

274 0.224825397134 0.74save best model for:  0.224825

2009 --> two thousand seven <==> two thousand nine

275 0.272238135338 0.89
September 10, 1881 --> september eleventh twenty thirteen <==> september tenth eighteen eighty one

276 0.270738095045 0.82
November 2012 --> november twenty fourteen <==> november twenty twelve

277 0.26570636034 0.9
1994 --> ninete

351 0.158273816109 0.6save best model for:  0.158274

December 30, 2419 --> december twenty first twenty fifteen <==> december thirtieth twenty four nineteen

352 0.189440473914 0.66
1 March 2016 --> the eleventh of may twenty eleven <==> the first of march twenty sixteen

353 0.218829363585 0.81
390 BCE. --> the fourteenth of june <==> three ninety b c e

354 0.178242072463 0.67
September 28, 1861 --> september twenty fourth twenty eleven <==> september twenty eighth eighteen sixty one

355 0.200912728906 0.73
2006 --> two thousand <==> two thousand six

356 0.216599211097 0.78
1959 --> nineteen ninety two <==> nineteen fifty nine

357 0.229138895869 0.77
1884 --> eighteen eighty four <==> eighteen eighty four

358 0.195781752467 0.71
2001 --> two thousand one <==> two thousand one

359 0.234261900187 0.8
2008-05-25 --> the twenty fourth of april two thousand nine <==> the twenty fifth of may two thousand eight

360 0.218888908625 0.75
April 1, 2010 --> april fourth twenty ten <==> ap

439 0.163345247507 0.58
25 May 2006 --> the twenty seventh of july two thousand eight <==> the twenty fifth of may two thousand six

440 0.142956346273 0.49
2006 --> two thousand four <==> two thousand six

441 0.159777790308 0.57
2003 --> two thousand three <==> two thousand three

442 0.139019846916 0.49
2007 --> two thousand seven <==> two thousand seven

443 0.173107162118 0.61
1970 --> nineteen seventy <==> nineteen seventy

444 0.152714297175 0.56
17 August --> the first of september <==> the seventeenth of august

445 0.151285722852 0.54
June 26 --> june twenty <==> june twenty sixth

446 0.132583335042 0.45
1833 --> eighteen thirty three <==> eighteen thirty three

447 0.163876995444 0.55
1869 --> eighteen sixty nine <==> eighteen sixty nine

448 0.143214285374 0.54
18 February 2016 --> the eighteenth of december twenty fifteen <==> the eighteenth of february twenty sixteen

449 0.201646819711 0.64
1877 --> eighteen seventy seven <==> eighteen seventy seven

450 0.147357150912 

...

6564 0.0169404763728 0.24
1900 --> nineteen hundred <==> nineteen hundred

6565 0.0165277775377 0.22
1928 --> nineteen twenty eight <==> nineteen twenty eight

6566 0.00333333341405 0.14
2012 --> twenty twelve <==> twenty twelve

6567 0.00972222257406 0.13
March 23, 2015 --> march twenty third twenty fifteen three thirteen three three three three thirteen three three three three three three three thirteen three <==> march twenty third twenty fifteen

6568 0.0209642872214 0.27
February 3, 2006 --> february third two thousand six <==> february third two thousand six

6569 0.0175079368055 0.17
1284 --> twelve eighty four <==> twelve eighty four

6570 0.0290000010282 0.19
2012 --> twenty twelve <==> twenty twelve

6571 0.0112301586196 0.23
1925 --> nineteen twenty five three three three three three three three three three three three <==> nineteen twenty five

6572 0.00509523786604 0.15
1942 --> nineteen forty two <==> nineteen forty two

6573 0.0145793650299 0.26
16 November 2006 --> the 

6726 0.0131111107767 0.2
2014 --> twenty fourteen <==> twenty fourteen

6727 0.00769444415346 0.18
1950s --> nineteen fifties <==> nineteen fifties

6728 0.0152341267094 0.15
February 10, 2011 --> february tenth twenty eleven three <==> february tenth twenty eleven

6729 0.00831349287182 0.23
1993 --> nineteen ninety three three three three three three three three three three three three three three three three three three three three three three three three three three three <==> nineteen ninety three

6730 0.0010000000475 0.2
4 August 1987 --> the fourth of august nineteen eighty seven <==> the fourth of august nineteen eighty seven

6731 0.0130793657154 0.14
1951 --> nineteen fifty one three three three three three three three three three three three three three three three three three three three three three three three three three three three three three <==> nineteen fifty one

6732 0.0140555556864 0.21
1824 --> eighteen twenty four <==> eighteen twenty four

6733 0.0112619046122

6893 0.0158690474927 0.17
1956 --> nineteen fifty six <==> nineteen fifty six

6894 0.00766666699201 0.14
1992 --> nineteen ninety two <==> nineteen ninety two

6895 0.00870634894818 0.15
2000 --> two thousand <==> two thousand

6896 0.0122777782381 0.22
2013 --> twenty thirteen <==> twenty thirteen

6897 0.00111111113802 0.12
10 January 2013 --> the tenth of january twenty thirteen three <==> the tenth of january twenty thirteen

6898 0.0146111110225 0.17
January 29, 1978 --> january twenty ninth nineteen seventy eight <==> january twenty ninth nineteen seventy eight

6899 0.00880952458829 0.08
28 February 2008 --> the twenty eighth of february two thousand eight <==> the twenty eighth of february two thousand eight

6900 0.0116388890892 0.11
May 16 --> may sixteenth <==> may sixteenth

6901 0.0102182542905 0.14
1877 --> eighteen seventy seven <==> eighteen seventy seven

6902 0.00914682541043 0.11
June 1954 --> june nineteen fifty six <==> june nineteen fifty four

6903 0.02160317450

6979 0.0118730161339 0.06
May 24, 1861 --> may twenty fourth nineteen seventy three <==> may twenty fourth eighteen sixty one

6980 0.0120833329856 0.1
1971 --> nineteen seventy one <==> nineteen seventy one

6981 0.00488888891414 0.07
2001 --> two thousand one <==> two thousand one

6982 0.0108730169013 0.12
1864 --> eighteen sixty four <==> eighteen sixty four

6983 0.0105238100514 0.08
1964 --> nineteen sixty four <==> nineteen sixty four

6984 0.0215952396393 0.21
March 16, 2005 --> march sixteenth two thousand five <==> march sixteenth two thousand five

6985 0.0137499999255 0.11
1846 --> eighteen forty six <==> eighteen forty six

6986 0.00336111127399 0.07
28 November 1982 --> the twenty eighth of november nineteen eighty two <==> the twenty eighth of november nineteen eighty two

6987 0.0136388884857 0.15
1994-11-24 --> the twenty fourth of december nineteen eighty one <==> the twenty fourth of november nineteen ninety four

6988 0.00739682558924 0.05
1903 --> nineteen o three 

KeyboardInterrupt: 